<a href="https://colab.research.google.com/github/jonghhhh/bert_inference_042124/blob/main/colab_selenium_%EB%84%A4%EC%9D%B4%EB%B2%84%EC%B9%B4%ED%8E%98%EB%B8%94%EB%A1%9C%EA%B7%B8_032424.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## selenium과 필요 라이브러리 불러오기

https://www.kaggle.com/code/yingfu46/scraping-a-dynamic-website-with-chrome-in-colab

In [ ]:
# Set up for running selenium in Google Colab
## You don't need to run this code if you do it in Jupyter notebook, or other local Python setting
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

In [ ]:
!pip install chromedriver-autoinstaller

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import chromedriver_autoinstaller

# setup chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()

# set up the webdriver
wd = webdriver.Chrome(options=chrome_options)

## 웹페이지에 접속하고 화면 하단까지 스크롤 다운하는 함수 만들기

In [ ]:
import urllib.parse
import pandas as pd
import time

def connect_scrolldown(target_url):
    try:
        wd.get(target_url)
        # 페이지 끝까지 스크롤
        last_height = wd.execute_script("return document.body.scrollHeight") # JavaScript 실행, 웹 페이지의 전체 높이를 반환
        while True:
            # 페이지 끝까지 스크롤 다운
            wd.execute_script("window.scrollTo(0, document.body.scrollHeight);") # 수평 스크롤 위치=0, 수직 스크롤 위치=document.body.scrollHeight(현재 페이지의 전체 높이=가장 하단)
            # 페이지 로드를 기다림
            time.sleep(2)  # 로딩 시간에 따라 조절이 필요할 수 있습니다.
            # 새로운 스크롤 높이를 계산하고 마지막 스크롤 높이와 비교
            new_height = wd.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        print("scroll-down completed")
    except Exception as e:
        print(f"An error occurred: {e}")

## 검색 조건 설정하고 내용 수집하는 함수 만들기

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import urllib.parse
import pandas as pd
import time

def navercafeblog_selenium_scrape(cafe=True, query='파이썬', date_from='20240301', date_to='20240302'):
    encoded_query=urllib.parse.quote(query)
    if cafe:
        target_url=f"https://search.naver.com/search.naver?ssc=tab.cafe.all&query={encoded_query}&cafe_where=articleg&st=rel&nso=so%3Ar%2Cp%3Afrom{date_from}to{date_to}"
    else:
        target_url=f"https://search.naver.com/search.naver?ssc=tab.blog.all&query={encoded_query}&cafe_where=articleg&st=rel&nso=so%3Ar%2Cp%3Afrom{date_from}to{date_to}"
    connect_scrolldown(target_url)
    time.sleep(1)
    html = wd.page_source
    time.sleep(0.5)
    soup = BeautifulSoup(html, 'html.parser')
    time.sleep(0.5)
    result_list=[]
    try:
        results = soup.select('div.view_wrap')
        for result in results:
            try: name=result.select('a.name')[0].text
            except: name=''
            try: title=result.select('a.title_link')[0].text
            except: title=''
            try: url=result.select('a.title_link')[0]['href']
            except: url=''
            try: date=result.select('span.sub')[0].text
            except:date=''
            try: text=result.select('a.dsc_link')[0].text
            except: text=''
            result_list.append([name,title,date,url,text])
    except Exception as e:
        print("An error occurred:", e)
    return pd.DataFrame(result_list, columns=['name','title','date','url','text'])

In [ ]:
# 실습
result_df=navercafeblog_selenium_scrape(cafe=False, query='총선', date_from='20240301', date_to='20240320') #True= 네이버 카페, False=네이버 블로그, 관련도 순으로 1000여개 추출
result_df.to_excel("--path/file.xlsx--")